# MGMTSA408 -- Lecture 2: Single-Leg Revenue Management

In this notebook, we will explore how to formulate the single-leg capacity control problem using linear programming.

## Formulating the static single-leg capacity control problem

Let us assume that we are managing seats on a flight on a Boeing 737-800, with ten fare types. The flight departs in 200 days from now. 

Here is the data we are given:


|Fare|Price|Probability|
|-|-|-|
|1|100|0.16|
|2|115|0.16| 
|3|120|0.14| 
|4|140|0.10| 
|5|150|0.10|
|6|210|0.08|
|7|220|0.06| 
|8|400|0.05| 
|9|450|0.04| 
|10|500|0.02|

The column labelled "Price" tells us how much one ticket of the given fare type costs, while the column labeled "Probability" tells us the probability of a customer requesting a fare of each type in a given period. We assume that we will observe at most one customer arrival in each period. 

Note that the probabilities do not add up to 1; they add up to 0.91. The remaining probability of 0.09 corresponds to not having any customer request in a period. 

Let's first create this data:

In [3]:
import numpy as np

# The number of fares
nFares = 10

# Revenue for each fare type:
revenue = np.array([100, 115, 120, 140, 150, 210, 220, 400, 450, 500])

# Probability of each fare type: 
probability = np.array([0.16, 0.16, 0.14, 0.1, 0.1, 0.08, 0.06, 0.05, 0.04, 0.02])

# Capacity of the flight: 
B = 160

# Time horizon:
T = 200

# Finally, let us compute the forecasted demand for each fare type:
forecast = T * probability
print(forecast)

sum(probability)
sum(forecast)

[32. 32. 28. 20. 20. 16. 12. 10.  8.  4.]


182.0

Now, let's go ahead with formulating the LP:

In [4]:
from gurobipy import *

# Create the model
m = Model()

# Create variables.
x = m.addVars(nFares, lb = 0, ub = forecast)

# Create the seat constraint
seat_constr = m.addConstr( sum(x[i] for i in range(nFares)) <= B )

# Set the objective.
m.setObjective(sum( revenue[i] * x[i] for i in range(nFares)), GRB.MAXIMIZE )

# Update + solve:
m.update()
m.optimize()

# Get the objective value
LP_obj = m.objval

# Get the allocation
allocation = np.array([x[i].x for i in range(nFares)])

# Display the results:
print("Allocation, forecast, objective:")
print(allocation)
print(forecast)
print(LP_obj)

sum(forecast)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 10 columns and 10 nonzeros
Model fingerprint: 0x3e59c441
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 5e+02]
  Bounds range     [4e+00, 3e+01]
  RHS range        [2e+02, 2e+02]
Presolve time: 0.01s
Presolved: 1 rows, 10 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+04   3.900000e+01   0.000000e+00      0s
       1    2.9440000e+04   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.944000000e+04
Allocation, forecast, objective:
[10. 32. 28. 20. 20. 16. 12. 10.  8.  4.]
[32. 32. 28. 20. 20. 16. 12. 10.  8.  4.]
29440.0


182.0

In the optimal solution to the static capacity control problem, we obtain a revenue of $29,440. The optimal allocation involves serving all of the demand except for the first fare type, which has the lowest price. 

## Solving the dynamic capacity control problem

In practice, we have to decide whether to accept or reject requests for seats in real time. Let us now see how we can use the capacity control LP above to solve the dynamic capacity control problem.

First, let us create a function that will solve the LP with a given number of available seats $b$ and for a given period $t$, and return the shadow price of the seat constraint. 

In [5]:
# Create the function bpc (= bid price control):
def bpc(b,t):
    seat_constr.rhs = b
    
    for i in range(nFares):
        x[i].ub = (T - t)*probability[i] #The -1 is excluded because we are in the 0-indexed world
    
    m.update()
    m.optimize()
    
    dual_value = seat_constr.pi
    
    return dual_value


# Test it out:
d1 = bpc(160, 1)
d2 = bpc(160, 100)
d3 = bpc(20, 100)

print("d1: ", d1)
print("d2: ", d2)
print("d3: ", d3)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 10 columns and 10 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 5e+02]
  Bounds range     [4e+00, 3e+01]
  RHS range        [2e+02, 2e+02]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.9372800e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  2.937280000e+04
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 10 columns and 10 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 5e+02]
  Bounds range     [2e+

Notice that we still see the output from Gurobi. We can turn this off by setting a parameter of the model:

In [6]:
# Set the OutputFlag parameter to 0 to disable logging
m.Params.outputflag = 0

# Now try the above code again:
d1 = bpc(160, 2)
d2 = bpc(100, 5)

print("d1: ", d1)
print("d2: ", d2)

# No solver logs anymore!

# Test it out:
d1 = bpc(160, 1)
d2 = bpc(160, 100)
d3 = bpc(20, 100)

print("d1: ", d1)
print("d2: ", d2)
print("d3: ", d3)

d1:  100.0
d2:  120.0
d1:  100.0
d2:  0.0
d3:  210.0


We now need to create a simulation for the policy corresponding to `bpc()`. There are several steps to go through here.

First, we will need to sample from a probability mass function to capture what happens in each period -- either we get a request for one of the 10 fares, or we do not get any request. We thus need to set up a proper probability mass function:

In [7]:
sum(probability)

0.9100000000000001

In [9]:
# Create a new array, probability_aug, which corresponds to a true probability
# mass function:
probability_aug = np.zeros(nFares+1)
probability_aug[0:nFares] = probability # First nFares elements are the same as probability
probability_aug[nFares] = 1 - sum(probability) # Last element is one minus the rest.

# Check that it is correct:
print(probability)
print(probability_aug)
print(sum(probability_aug))

[0.16 0.16 0.14 0.1  0.1  0.08 0.06 0.05 0.04 0.02]
[0.16 0.16 0.14 0.1  0.1  0.08 0.06 0.05 0.04 0.02 0.09]
1.0


Next, we need a loop to simulate the request arrivals. 

In [10]:
arrival_sequence = np.random.choice(range(nFares+1), T, p=probability_aug)
print(arrival_sequence)

[ 3  4  0  2  1 10  2  6  4  0  1 10  4  1  2  2  6  0  1  0  7  0  2  0
  4  5  2  5  3  0  6  3  5  2  5  2  2 10  0  2  0  3  3  5  6  3  6  0
  0 10  1  0  9  3  6  4  4  0  3  7  4  2  7  2 10  2  8  7  8  0 10  3
  5  6  6  3  0  3 10  2  7  4  3  3 10 10 10  0 10  3  3  3  9  2  4  2
  0  1  0  5  0  2  2 10  7  5  3  1  2  1  1  0  1  2  3  0  3  3  1  5
 10  1  3  1  1 10  6  1  3  0  4  1  3  1  8  0  1  0  2  3  8 10  0  1
  0  5  0  0 10  0 10  4  3  7  0  8  0  4  3  4  9  3 10  1  7  7  7  6
  3  0  1  1 10  0  1  4  8  6  1  2  3  5  1  9  1  2  1  1 10  6  1  4
  3  9  1  2  5  8  0  2]


In [11]:
nSimulations = 100

np.random.seed(101)

results_revenue = np.zeros(nSimulations)
results_remaining_seats = np.zeros(nSimulations)

for s in range(nSimulations):
    total_revenue = 0.0
    b = B
    arrival_sequence = np.random.choice(range(nFares+1), T, p=probability_aug)
    
    for t in range(T):
        if (b == 0):
            break
        
        fare = arrival_sequence[t]
        
        if (fare < nFares):
            dual_val = bpc(b,t)
            if (revenue[fare] >= dual_val):
                # Accept the fare!
                b -= 1
                total_revenue += revenue[fare]
    
    results_revenue[s] = total_revenue
    results_remaining_seats[s] = b
    

mean_LP_revenue = results_revenue.mean()
mean_LP_remaining_seats = results_remaining_seats.mean()
    
print("Mean revenue: ", mean_LP_revenue)
print("Mean seats remaining: ", mean_LP_remaining_seats)

Mean revenue:  29329.25
Mean seats remaining:  0.04


The optimal expected revenue is upper bounded by the optimal value of the static capacity control LP. We can thus compare the above revenue to the LP objective from earlier to see how close we are to optimality:

In [9]:
suboptimality_gap = 100*( (LP_obj - mean_LP_revenue) / LP_obj)
print("Suboptimality gap: ", suboptimality_gap)

Suboptimality gap:  0.37618885869565216


In other words: we are at most 0.4% away from the best possible expected revenue!

## Simulating a myopic policy

As a comparison, let us consider a different kind of policy. The following policy involves basically accepting any request, so long as there is at least one seat remaining. This type of policy is called a "myopic" policy because it does not take into account what types of requests may come in the future. 

(In contrast, the dual values from our LP-based policy are derived by accounting for how many seats remain, and how many requests of each fare type are expected in the future.)

In [10]:
nSimulations = 100

np.random.seed(101)

results_myopic_revenue = np.zeros(nSimulations)
results_myopic_remaining_seats = np.zeros(nSimulations)

for s in range(nSimulations):
    total_revenue = 0
    b = B
    arrival_sequence = np.random.choice(range(nFares+1), T, p=probability_aug)
    
    for t in range(T):
        if (b == 0):
            break
        
        fare = arrival_sequence[t]
        
        if (fare < nFares):
            # Immediately accept the fare (compare to LP-based policy above)
            b -= 1
            total_revenue += revenue[fare]
    
    results_myopic_revenue[s] = total_revenue
    results_myopic_remaining_seats[s] = b

    
mean_myopic_revenue = results_myopic_revenue.mean()
mean_myopic_remaining_seats = results_myopic_remaining_seats.mean()
    
print("Mean revenue (myopic): ", mean_myopic_revenue)
print("Mean seats remaining (myopic): ", mean_myopic_remaining_seats)
print("Expected number of arrivals of any kind: ", T * sum(probability))

Mean revenue (myopic):  27787.7
Mean seats remaining (myopic):  0.0
Expected number of arrivals of any kind:  182.00000000000003


Let's compute the improvement in the average revenue:

In [11]:
print("Average improvement: ", 100*(mean_LP_revenue/mean_myopic_revenue - 1))
print("Average improvement: ", 100*(mean_LP_revenue - mean_myopic_revenue) / mean_myopic_revenue)

Average improvement:  5.5475983978522825
Average improvement:  5.547598397852285


Notice that the myopic policy is substantially worse compared to our LP-based policy; our LP-based policy achieves about a 5.5% improvement over the myopic policy.

In [12]:
cost = 26000
expected_profit_LP = mean_LP_revenue - cost
expected_profit_myopic = mean_myopic_revenue - cost
profit_improvement = (expected_profit_LP - expected_profit_myopic) / expected_profit_myopic * 100
print("Exp. profit of LP-based policy: ", expected_profit_LP)
print("Exp. profit of myopic policy: ", expected_profit_myopic)
print("Exp. profit improvement: ", profit_improvement)

Exp. profit of LP-based policy:  3329.25
Exp. profit of myopic policy:  1787.7000000000007
Exp. profit improvement:  86.23091122671582
